# 1. 얼굴 인식

In [ ]:
import cv2
import face_recognition
 
# 이미지 불러오기
image_to_be_matched = face_recognition.load_image_file('./img/forproject/jihnotestface.png')  # 자신의 얼굴사진
name = "Jinho Kim" # 화면에 뜰 이름
 
# 이미지 인코딩
image_to_be_matched_encoded = face_recognition.face_encodings(image_to_be_matched)[0]
print(image_to_be_matched_encoded)
 

cap = cv2.VideoCapture(0)
 
if not cap.isOpened():  # 카메라가 없을때
    print("no camera")
    exit()
    

# 프로그램 실행 
while cap.isOpened():
    status, frame = cap.read()
 
    if not status:
        print("Could not read frame")
        exit()
 
    # CNN 기반 얼굴 검출
    face_locations = face_recognition.face_locations(frame, number_of_times_to_upsample=0, model="cnn")
 
    for face_location in face_locations:
        # 얼굴 위치 확인
        top, right, bottom, left = face_location
        face_image = frame[top:bottom, left:right]
        # 얼굴 검출 및 이름 확인
        try:
            face_encoded = face_recognition.face_encodings(face_image)[0]
            result = face_recognition.compare_faces([image_to_be_matched_encoded], face_encoded, 0.5)
 
            if result[0] == True:
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                Y = top - 10 if top - 10 > 10 else top + 10
                text = name
                cv2.putText(frame, text, (left, Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        except:
            pass
    cv2.imshow("detect", frame)
 
    if cv2.waitKey(1) & 0xFF == 27:
        break


cap.release()
cv2.destroyAllWindows()


[-0.09088071  0.01222388  0.02374151 -0.00227743 -0.07055598 -0.04537076
 -0.02450605 -0.14982696  0.0941547  -0.11063514  0.25660756 -0.07492588
 -0.14040518 -0.12960839 -0.03632411  0.179598   -0.21625341 -0.0899235
 -0.10262206  0.02158342  0.09740109 -0.03866766  0.01573879  0.06442802
 -0.05511615 -0.35600311 -0.07439014 -0.0956791   0.0206669  -0.02324483
 -0.01261473  0.03858064 -0.17111205 -0.01612779 -0.01626258  0.04045497
  0.00170405 -0.06337783  0.16810632 -0.06889766 -0.23470351  0.01119609
  0.01624773  0.1812623   0.160974    0.03279953  0.00931055 -0.13287586
  0.16052768 -0.18219283 -0.00834361  0.13520071  0.08690301  0.02830932
  0.00871639 -0.06723566  0.04385594  0.09087777 -0.1439651   0.00104487
  0.05027964 -0.10891877 -0.055757   -0.1212797   0.19228953  0.09449381
 -0.12290385 -0.169615    0.0934263  -0.13088344 -0.08878209  0.01622056
 -0.14125019 -0.19124308 -0.34244075 -0.01155382  0.39493835  0.07133129
 -0.22238548  0.0493352  -0.07459068  0.01903202  0.

# 2. 코 검출

In [ ]:
import cv2
import numpy as np

# 코 검출 데이터
nose_cascade = cv2.CascadeClassifier('./haarcascade_mcs_nose.xml')
mouth_cascade = cv2.CascadeClassifier('./haarcascade_mcs_mouth.xml')

if nose_cascade.empty():
    raise IOError('Cannot find nose cascade classifier xml file!')
if mouth_cascade.empty():
    raise IOError('Cannot find mouth cascade classifier xml file!')

cap = cv2.VideoCapture(0)
ds_factor = 1.0

# 프로그램 실행
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=ds_factor, fy=ds_factor, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 코 검출
    nose_rects = nose_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in nose_rects:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)
        cv2.putText(frame, "please wear your mask", (20,50), cv2.FONT_HERSHEY_DUPLEX, 1.5, (0,0,255))
        break

    cv2.imshow('Nose Detector', frame)
    key = cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

# 3. 얼굴인식, 코,입 검출시 경고메세지 (2번 코검출 개선)

In [3]:
import cv2
import numpy as np

# 얼굴, 코, 입 검출을 위한 케스케이드 분류기 생성
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
nose_cascade = cv2.CascadeClassifier('./haarcascade_mcs_nose.xml')
mouth_cascade = cv2.CascadeClassifier('./haarcascade_mcs_mouth.xml')

if nose_cascade.empty():
    raise IOError('Cannot find nose cascade classifier xml file!')
if mouth_cascade.empty():
    raise IOError('Cannot find mouth cascade classifier xml file!')

cap = cv2.VideoCapture(0)

# 프로그램 실행
while cap.isOpened():
    ret, img = cap.read()
    if ret:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 얼굴 검출
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(80,80))
        for (x,y,w,h) in faces:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0),2)
            roi = gray[y:y+h, x:x+w]
            
            # 코 검출
            nose = nose_cascade.detectMultiScale(roi)
            for i, (nx,ny,nw,nh) in enumerate(nose):
                if i>=2:
                    break
                cv2.rectangle(img[y:y+h, x:x+w],(nx,ny), (nx+nw,ny+nh), (0,0,255),2)
                cv2.putText(img, "please wear your mask", (20,50), cv2.FONT_HERSHEY_DUPLEX, 1.5, (0,0,255))
            # 입 검출
            mouth = mouth_cascade.detectMultiScale(roi)
            for i, (mx,my,mw,mh) in enumerate(nose):
                if i>=2:
                    break
                cv2.rectangle(img[y:y+h, x:x+w],(mx,my), (mx+mw,my+mh), (0,0,255),2)
                cv2.putText(img, "please wear your mask", (20,50), cv2.FONT_HERSHEY_DUPLEX, 1.5, (0,0,255))
        cv2.imshow('Detector', img)
    else:
        break
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()